In [1]:
import warnings
warnings.filterwarnings('ignore')

## 1. Pinecone

In [2]:
%%capture
! pip install pinecone

In [3]:
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = ""

pc = Pinecone(api_key=pinecone_api_key)

pc.create_index("quickstart", spec=ServerlessSpec("aws", "us-east-1"), dimension=1536, metric="euclidean")
pinecone_index = pc.Index("quickstart")

In [4]:
%%capture
! pip install datasets

In [5]:
from datasets import load_dataset

dataset = load_dataset('klue', 'mrc', split='train')

In [6]:
%%capture
! pip install llama-index

In [7]:
from llama_index.core import Document

text_list = dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

In [8]:
%%capture
! pip install llama-index-vector-stores-pinecone

In [9]:
import os
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

os.environ["OPENAI_API_KEY"] = ""

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context
)

Upserted vectors:   0%|          | 0/130 [00:00<?, ?it/s]

In [12]:
print(dataset[0]['question'])

retrieval_engine = index.as_retriever(similarity_top_k=5, verbose=True)
response = retrieval_engine.retrieve(
    dataset[0]['question']
)
print(len(response))
print(response[0].node.text)

북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
4
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.
